<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DM_GEMINI_DEMO_models_gemini_2_5_pro_preview_05_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-generativeai -q

In [2]:
from google.colab import userdata
import google.generativeai as genai


from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

import google.generativeai as genai

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp

In [5]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import networkx as nx
from datetime import datetime
from typing import List, Dict, Tuple, Any, Optional
from collections import deque
import random
import google.generativeai as genai
from google.colab import userdata  # This is needed for accessing user data in Google Colab

#   Gemini LLM Interface
class GeminiLLMInterface:
    """Handles communication with the Gemini LLM."""

    def __init__(self, model_name: str = "models/gemini-2.5-pro-preview-05-06"):
        """Initializes the interface with the specified Gemini model."""

        self.model_name = model_name
        self.gemini_client: Optional[Any] = None
        print("Attempting to get Gemini API key...")
        print(f"Model: {model_name}")
        print('\n\n')
        try:
            # Get API key from Colab userdata
            GOOGLE_API_KEY = userdata.get('GEMINI')
            #print("Gemini API Key:", GOOGLE_API_KEY) # Print to verify
            genai.configure(api_key=GOOGLE_API_KEY)
            self.gemini_client = genai
        except ImportError:
            print("Warning: google.colab not available. Running in local mode.")

            class DummyGeminiClient:  # Define a dummy client for local execution
                class GenerativeModel:
                    def __init__(self, model_name):
                        self.model_name = model_name

                    def generate_content(self, *args, **kwargs):
                        class Response:
                            def __init__(self):
                                self.text = "Dummy Gemini response"

                        return Response()

                types = None
                Part = None

            self.gemini_client = DummyGeminiClient()


    def query_gemini(self, prompt: str, data: Optional[Dict] = None,
                      code_execution: bool = False, model_name: Optional[str] = None) -> str:
        """Queries the Gemini model with the given prompt and data."""

        effective_model_name = model_name if model_name else self.model_name

        print(f"--- Gemini Prompt ({effective_model_name}): ---\n{prompt}\n---")
        if data is not None:
            print("--- Gemini Data: ---\n", data, "\n---")

        if self.gemini_client:
            try:
                model = self.gemini_client.GenerativeModel(effective_model_name)
                generation_config = self.gemini_client.types.GenerationConfig()
                if data is not None:
                    response = model.generate_content(
                        parts=[{"text": prompt}, self.gemini_client.Part.from_data(data=data,
                                                                                  mime_type="application/json")],
                        generation_config=generation_config
                    ).text
                else:
                    response = model.generate_content(
                        prompt,
                        generation_config=generation_config
                    ).text
                return response
            except Exception as e:
                print(f"Error querying Gemini: {e}")
                return "Error: Could not retrieve Gemini response."
        else:
            return "Dummy response (Gemini not available)"


class FlightDisruptionAgent:
    """
    An AI agent for intelligent flight disruption management,
    specifically leveraging Gemini models.
    """

    def __init__(self, llm_model: str = "models/gemini-2.5-pro-preview-05-06"):
        """Initializes the agent with data structures and AI models."""

        self.gemini_llm = GeminiLLMInterface(llm_model)
        self.flights = pd.DataFrame()  # Flight schedule data
        self.aircraft = pd.DataFrame()  # Aircraft information
        self.crew = pd.DataFrame()  # Crew schedules and qualifications
        self.airports = pd.DataFrame()  # Airport capacities and constraints
        self.realtime_data = deque(maxlen=1000)  # Recent flight updates
        # self.delay_predictor = self.build_delay_predictor()  # Moved to load_initial_data
        self.optimizer = self.build_optimizer()  # Optimization algorithm
        self.impact_analyzer = self.build_impact_analyzer()  # Network analysis tool

        self.disruption_threshold = 5  # Delay threshold (minutes) - Lowered to 10

        #self.disruption_threshold = 4
        self.prediction_horizon = 120  # Timeframe for delay prediction (minutes)
        self.recovery_window = 240  # Timeframe for recovery planning (minutes)

    def load_initial_data(self, flights: pd.DataFrame, aircraft: pd.DataFrame,
                           crew: pd.DataFrame, airports: pd.DataFrame) -> None:
        """Loads the initial flight schedule and related data."""

        self.flights = flights
        self.aircraft = aircraft
        self.crew = crew
        self.airports = airports

        # Added: Prepare the data for delay prediction
        weather_codes = {'Clear': 0, 'Rain': 1, 'Snow': 2, 'Fog': 3, 'Storm': 4}
        self.flights['weather_condition_encoded'] = self.flights['weather_condition'].map(weather_codes).fillna(-1)
        self.flights['scheduled_departure_time_of_day'] = (self.flights['scheduled_departure_time'].dt.hour * 60 +
                                                          self.flights['scheduled_departure_time'].dt.minute)
        self.flights['previous_delay_minutes'] = 0  # Initialize with 0

        # Add departure and arrival airport capacities to the flights DataFrame
        self.flights = pd.merge(self.flights, self.airports[['airport_code', 'capacity']],
                              left_on='departure_airport', right_on='airport_code',
                              how='left').rename(columns={'capacity': 'departure_airport_capacity'})
        self.flights = pd.merge(self.flights, self.airports[['airport_code', 'capacity']],
                              left_on='arrival_airport', right_on='airport_code',
                              how='left').rename(columns={'capacity': 'arrival_airport_capacity'})
        self.flights = self.flights.drop(['airport_code_x', 'airport_code_y'], axis=1)  # Drop unnecessary columns

        # --- Introduce Artificial Delays for Training ---
        delay_probability = 0.2  # Probability of a flight being delayed (adjust as needed)
        max_delay = 60  # Maximum artificial delay in minutes (adjust as needed)

        num_flights_to_delay = int(len(self.flights) * delay_probability)
        delayed_flight_indices = random.sample(self.flights.index.tolist(), num_flights_to_delay)

        for index in delayed_flight_indices:
            self.flights.loc[index, 'delay_minutes'] = random.randint(1, max_delay)
        # --- End of Artificial Delay Introduction ---

        # Now it is safe to build the delay predictor
        self.delay_predictor = self.build_delay_predictor()

    def update_realtime_data(self, updates: List[Dict[str, Any]]) -> None:
        """Updates the agent with real-time flight schedule updates."""
        for update in updates:
            self.realtime_data.append(update)

    def build_delay_predictor(self) -> RandomForestRegressor:
        """Builds a Random Forest model to predict flight delays."""

        features = ['scheduled_departure_time_of_day', 'departure_airport_capacity',
                    'arrival_airport_capacity', 'previous_delay_minutes', 'weather_condition_encoded']
        target = 'delay_minutes'

        # Prepare training data
        training_data = self.flights[features + [target]].dropna()
        X = training_data[features]
        y = training_data[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the Random Forest model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        return model

    def build_optimizer(self) -> Any:  # More specific type hint later
        """Builds a simplified recovery optimizer."""

        def optimizer(flights: pd.DataFrame, disruptions: Dict[str, float],
                      aircraft: pd.DataFrame, crew: pd.DataFrame,
                      airports: pd.DataFrame) -> Dict[str, Dict[str, Any]]:
            """Finds recovery actions based on a simplified logic."""
            recovery_actions = {}
            for flight_id, delay in disruptions.items():
                flight = flights.loc[flights['flight_id'] == flight_id].iloc[0]  # Get flight details
                recovery_actions[flight_id] = {}  # Initialize action dict

                if delay > 120:  # If delay is too long, cancel
                    recovery_actions[flight_id]['action'] = 'cancel'
                else:
                    recovery_actions[flight_id]['action'] = 'delay'
                    recovery_actions[flight_id]['delay_time'] = delay

                    # Simplified resource reassignment (Replace with proper optimization)
                    available_aircraft = aircraft[aircraft['aircraft_type'] == flight['aircraft_type']]
                    available_crew = crew[crew['aircraft_type_qualification'].apply(
                        lambda x: flight['aircraft_type'] in x)]

                    if not available_aircraft.empty:
                        recovery_actions[flight_id]['new_aircraft'] = random.choice(
                            available_aircraft['aircraft_id'].tolist())

                    if not available_crew.empty:
                        recovery_actions[flight_id]['new_crew'] = random.choice(available_crew['crew_id'].tolist())

            return recovery_actions

        return optimizer

    def build_impact_analyzer(self) -> Any:  # More specific type hint later
        """Builds a network-based impact analyzer."""

        def analyzer(flights: pd.DataFrame, disruptions: Dict[str, float]) -> Tuple[
            Dict[str, float], Dict[str, str]]:
            """Analyzes the impact of disruptions using a flight network."""
            G = nx.DiGraph()  # Create a directed graph
            for index, row in flights.iterrows():
                G.add_edge(row['departure_airport'], row['arrival_airport'],
                           weight=row['flight_time'])  # Add flight connections

            impacted_flights = {}
            llm_explanations = {}
            for flight_id, delay in disruptions.items():
                # Simplified: Propagate delay to the next flight in the schedule
                # Changed: Use .loc with flight_id column instead of index
                next_flight_id = flights[flights['departure_airport'] == flights.loc[flights['flight_id'] == flight_id, 'arrival_airport'].iloc[0]]['flight_id'].values
                if next_flight_id.size > 0:
                    next_flight_id = next_flight_id[0]  # Assume only one next flight for simplicity
                    impacted_flights[next_flight_id] = impacted_flights.get(next_flight_id, 0) + delay
                    llm_explanations[next_flight_id] = f"Potential delay propagation from flight {flight_id}"
            return impacted_flights, llm_explanations

        return analyzer

    def process_realtime_updates(self, updates: List[Dict[str, Any]]) -> None:
        """Processes real-time flight updates and manages disruption recovery."""
        self.update_realtime_data(updates)  # Store the updates
        disruptions = self.detect_disruptions()  # Identify disruptions
        impacted_flights, impact_explanations = self.impact_analyzer(self.flights,
                                                                       disruptions)  # Analyze impact
        recovery_options = self.optimizer(self.flights, disruptions, self.aircraft, self.crew,
                                         self.airports)  # Generate recovery plan
        self.communicate_disruptions(self.flights, disruptions, impact_explanations, recovery_options,
                                     updates)  # Communicate
        self.learn_from_outcome(disruptions, recovery_options)  # Learn (Placeholder)

    def detect_disruptions(self) -> Dict[str, float]:
        """Detects flight disruptions by predicting delays."""
        disruptions = {}
        for index, flight in self.flights.iterrows():
            # Prepare feature vector for delay prediction
            # Changed: Use .loc with airport_code column
            departure_airport_capacity = self.airports.loc[self.airports['airport_code'] == flight['departure_airport'], 'capacity'].iloc[0]
            arrival_airport_capacity = self.airports.loc[self.airports['airport_code'] == flight['arrival_airport'], 'capacity'].iloc[0]

            features = {
                'scheduled_departure_time_of_day': (flight['scheduled_departure_time'].hour * 60 +
                                                      flight['scheduled_departure_time'].minute),
                'departure_airport_capacity': departure_airport_capacity,  # Use extracted value
                'arrival_airport_capacity': arrival_airport_capacity,  # Use extracted value
                'previous_delay_minutes': self.get_previous_delay(flight['flight_id']),
                'weather_condition_encoded': flight['weather_condition_encoded']
            }
            feature_vector = pd.DataFrame([features])
            predicted_delay = self.delay_predictor.predict(feature_vector)[0]  # Predict delay
            print(f"Flight {flight['flight_id']}: Predicted Delay = {predicted_delay}")  # Print predicted delay

            if predicted_delay > self.disruption_threshold:
                disruptions[flight['flight_id']] = predicted_delay
        return disruptions

    def get_previous_delay(self, flight_id: str) -> float:
        """Retrieves the delay of the previous flight of the same aircraft."""
        # Placeholder: Replace with actual logic to track aircraft routing and delays
        # This requires maintaining a history of aircraft assignments and delays
        return 0.0

    def get_current_weather(self, airport_code: str) -> str:
        """Retrieves the current weather condition at the specified airport."""
        # Placeholder: Replace with actual weather data integration
        # This would involve querying a weather API or database
        return "Clear"

    def communicate_disruptions(self, flights: pd.DataFrame,
                                disruptions: Dict[str, float],
                                impact_explanations: Dict[str, str],
                                recovery_options: Dict[str, Dict[str, Any]],
                                updates: List[Dict[str, Any]]) -> None:
        """Generates detailed, flight-specific assessment reports using Gemini."""
        report = "--- Flight Disruption Assessment Reports ---\n\n"  # Initialize the report

        for flight_id, delay in disruptions.items():
            flight_data = flights.loc[flights['flight_id'] == flight_id].to_dict(
                orient='records')[0]  # Get detailed flight data
            update_data = next(
                (update for update in updates if update['flight_id'] == flight_id),
                None)  # Find the relevant update

            if update_data:
                prompt = f"""
                    Analyze the potential impact of a {delay} minute disruption to Flight {flight_data['airline_code']} {flight_data['flight_number']}
                    (Departure: {flight_data['departure_airport']}, Arrival: {flight_data['arrival_airport']}, Scheduled Departure: {flight_data['scheduled_departure_time']}).

                    Consider the following factors:

                    **Flight Details:**
                    -   Aircraft type: {flight_data['aircraft_type']}
                    -   Passenger count: {flight_data['passenger_count']}
                    -   Estimated disruption cost: {update_data['disruption_cost']}

                    **Disruption Details:**
                    -   Disruption type: {update_data['type']}
                    -   Delay (minutes): {delay}

                    **Contextual Factors:**
                    -   Impact on connecting flights at {flight_data['arrival_airport']}.
                    -   Crew availability and duty time restrictions.
                    -   Gate availability at {flight_data['departure_airport']} and {flight_data['arrival_airport']}.

                    **Assessment Requirements:**
                    1.  **Summary of Key Risks and Challenges:** (e.g., passenger misconnections, crew legality issues).
                    2.  **Potential Recovery Options and Trade-offs:** (e.g., delaying the flight, swapping aircraft, reassigning crew, canceling the flight). For each option, discuss the pros, cons, and estimated cost.
                    3.  **Recommendation for the Best Course of Action:** Justify your recommendation based on minimizing overall disruption, cost-effectiveness, and passenger satisfaction.
                    4.  **Describe the potential impact of delaying this flight on subsequent flights in the schedule.** Consider factors such as connecting passenger itineraries, crew schedules, and aircraft availability.


                    **Output Format:**
                    Provide the report in a structured format with clear headings for each section (e.g., "1. Summary of Key Risks...").
                    """

                flight_assessment = self.gemini_llm.query_gemini(prompt,
                                                               code_execution=True)  # Get assessment from LLM
                report += f"\n--- Flight {flight_id} ({flight_data['airline_code']} {flight_data['flight_number']}) Assessment ---\n{flight_assessment}\n"
            else:
                report += f"\n--- Flight {flight_id} ({flight_data['airline_code']} {flight_data['flight_number']}) Assessment ---\nNo update data found for this flight.\n"

        print(report)  # Print the compiled report

    def learn_from_outcome(self, disruptions: Dict[str, float],
                            recovery_options: Dict[str, Dict[str, Any]]) -> None:
        """Placeholder for learning from the outcome of disruptions."""
        # This method would ideally update the delay prediction model,
        # refine the optimization strategies, and store successful recovery patterns
        # for future use.
        pass

In [6]:
#   Gemini : AI Agent for Intelligent Flight Disruption Management

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
import networkx as nx
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Any, Optional
from collections import deque
import random

from google.colab import userdata
import google.generativeai as genai

#   --- Example Usage ---
if __name__ == '__main__':
    #   Sample Data
    flights_data = pd.DataFrame({
        'flight_id': [1, 2, 3, 4, 5, 6, 7, 8],
        'airline_code': ['AA', 'DL', 'UA', 'AA', 'B6', 'WN', 'AS', 'AC'],
        'flight_number': [123, 456, 789, 101, 202, 303, 404, 505],
        'departure_airport': ['ATL', 'BOS', 'ORD', 'DFW', 'JFK', 'LAX', 'SEA', 'YYZ'],
        'arrival_airport': ['BOS', 'ORD', 'DFW', 'JFK', 'LAX', 'SEA', 'YYZ', 'ATL'],
        'scheduled_departure_time': [datetime(2024, 1, 1, 7, 0), datetime(2024, 1, 1, 9, 0),
                                     datetime(2024, 1, 1, 11, 0), datetime(2024, 1, 1, 13, 0),
                                     datetime(2024, 1, 1, 8, 30), datetime(2024, 1, 1, 10, 30),
                                     datetime(2024, 1, 1, 12, 30), datetime(2024, 1, 1, 14, 30)],
        'scheduled_arrival_time': [datetime(2024, 1, 1, 8, 0), datetime(2024, 1, 1, 10, 0),
                                   datetime(2024, 1, 1, 12, 0), datetime(2024, 1, 1, 14, 0),
                                   datetime(2024, 1, 1, 9, 30), datetime(2024, 1, 1, 11, 30),
                                   datetime(2024, 1, 1, 13, 30), datetime(2024, 1, 1, 15, 30)],
        'flight_time': [60, 60, 60, 60, 60, 60, 60, 60],
        'aircraft_type': ['737', '737', '777', '737', 'A320', '737', '737', '787'],
        'weather_condition': ['Clear', 'Rain', 'Clear', 'Snow', 'Fog', 'Clear', 'Rain', 'Clear'],
        'delay_minutes': [0, 0, 0, 0, 0, 0, 0, 0],
        'passenger_count': [120, 140, 250, 110, 160, 130, 100, 200]
    })
    aircraft_data = pd.DataFrame({
        'aircraft_id': ['AA1', 'AA2', 'BB1', 'CC1', 'DD1', 'EE1'],
        'aircraft_type': ['737', '777', '737', '777', 'A320', '787'],
        'capacity': [150, 300, 150, 300, 180, 250]
    })
    crew_data = pd.DataFrame({
        'crew_id': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'],
        'crew_base': ['ATL', 'BOS', 'ORD', 'DFW', 'JFK', 'LAX'],
        'aircraft_type_qualification': [['737'], ['737', '777'], ['777'], ['737', 'A320'], ['A320'], ['787']]
    })
    airports_data = pd.DataFrame({
        'airport_code': ['ATL', 'BOS', 'ORD', 'DFW', 'JFK', 'LAX', 'SEA', 'YYZ'],
        'capacity': [2, 3, 2, 1, 1, 3, 2, 2]
    })

    gemini_agent = FlightDisruptionAgent()
    gemini_agent.load_initial_data(flights_data.copy(), aircraft_data.copy(),
                                  crew_data.copy(), airports_data.copy())

    updates = [
        {'flight_id': 1, 'type': 'arrival_delay', 'delay': 20,
         'scheduled_departure_time': datetime(2024, 1, 1, 7, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 8, 0),
         'airline_code': 'AA', 'flight_number': 123, 'departure_airport': 'ATL',
         'arrival_airport': 'BOS', 'passenger_count': 120, 'disruption_cost': 5000},
        {'flight_id': 2, 'type': 'crew_delay', 'delay': 30,
         'scheduled_departure_time': datetime(2024, 1, 1, 9, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 10, 0),
         'airline_code': 'DL', 'flight_number': 456, 'departure_airport': 'BOS',
         'arrival_airport': 'ORD', 'passenger_count': 140, 'disruption_cost': 7000},
        {'flight_id': 3, 'type': 'departure_delay', 'delay': 10,
         'scheduled_departure_time': datetime(2024, 1, 1, 11, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 12, 0),
         'airline_code': 'UA', 'flight_number': 789, 'departure_airport': 'ORD',
         'arrival_airport': 'DFW', 'passenger_count': 250, 'disruption_cost': 12000},
        {'flight_id': 4, 'type': 'weather_update', 'airport': 'DFW',
         'weather_condition': 'Storm',
         'scheduled_departure_time': datetime(2024, 1, 1, 13, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 14, 0),
         'airline_code': 'AA', 'flight_number': 101, 'departure_airport': 'DFW',
         'arrival_airport': 'JFK', 'passenger_count': 110, 'disruption_cost': 4000},
        {'flight_id': 5, 'type': 'arrival_delay', 'delay': 15,
         'scheduled_departure_time': datetime(2024, 1, 1, 8, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 9, 30),
         'airline_code': 'B6', 'flight_number': 202, 'departure_airport': 'JFK',
         'arrival_airport': 'LAX', 'passenger_count': 160, 'disruption_cost': 6000},
        {'flight_id': 6, 'type': 'maintenance_delay', 'delay': 45,
         'scheduled_departure_time': datetime(2024, 1, 1, 10, 30),
         'scheduled_arrival_time': datetime(2024, 1, 11, 11, 30),
         'airline_code': 'WN', 'flight_number': 303, 'departure_airport': 'LAX',
         'arrival_airport': 'SEA', 'passenger_count': 130, 'disruption_cost': 9000},
        {'flight_id': 7, 'type': 'departure_delay', 'delay': 25,
         'scheduled_departure_time': datetime(2024, 1, 1, 12, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 13, 30),
         'airline_code': 'AS', 'flight_number': 404, 'departure_airport': 'SEA',
         'arrival_airport': 'YYZ', 'passenger_count': 100, 'disruption_cost': 3000},
        {'flight_id': 8, 'type': 'crew_delay', 'delay': 60,
         'scheduled_departure_time': datetime(2024, 1, 1, 14, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 15, 30),
         'airline_code': 'AC', 'flight_number': 505, 'departure_airport': 'YYZ',
         'arrival_airport': 'ATL', 'passenger_count': 200, 'disruption_cost': 10000},
        {'flight_id': 1, 'type': 'gate_change', 'airport': 'ATL', 'new_gate': 'A2',
         'scheduled_departure_time': datetime(2024, 1, 1, 7, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 8, 0),
         'airline_code': 'AA', 'flight_number': 123, 'departure_airport': 'ATL',
         'arrival_airport': 'BOS', 'passenger_count': 120, 'disruption_cost': 2000},
        {'flight_id': 3, 'type': 'weather_update', 'airport': 'ORD', 'weather_condition': 'Fog',
         'scheduled_departure_time': datetime(2024, 1, 1, 11, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 12, 0),
         'airline_code': 'UA', 'flight_number': 789, 'departure_airport': 'ORD',
         'arrival_airport': 'DFW', 'passenger_count': 250, 'disruption_cost': 8000},
        {'flight_id': 2, 'type': 'arrival_delay', 'delay': 10,
         'scheduled_departure_time': datetime(2024, 1, 1, 9, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 10, 0),
         'airline_code': 'DL', 'flight_number': 456, 'departure_airport': 'BOS',
         'arrival_airport': 'ORD', 'passenger_count': 140, 'disruption_cost': 3000},
        {'flight_id': 4, 'type': 'departure_delay', 'delay': 35,
         'scheduled_departure_time': datetime(2024, 1, 1, 13, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 14, 0),
         'airline_code': 'AA', 'flight_number': 101, 'departure_airport': 'DFW',
         'arrival_airport': 'JFK', 'passenger_count': 110, 'disruption_cost': 9000},
        {'flight_id': 6, 'type': 'crew_delay', 'delay': 15,
         'scheduled_departure_time': datetime(2024, 1, 1, 10, 30),
         'scheduled_arrival_time': datetime(2024, 1, 11, 11, 30),
         'airline_code': 'WN', 'flight_number': 303, 'departure_airport': 'LAX',
         'arrival_airport': 'SEA', 'passenger_count': 130, 'disruption_cost': 4000},
        {'flight_id': 8, 'type': 'arrival_delay', 'delay': 5,
         'scheduled_departure_time': datetime(2024, 1, 1, 14, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 15, 30),
         'airline_code': 'AC', 'flight_number': 505, 'departure_airport': 'YYZ',
         'arrival_airport': 'ATL', 'passenger_count': 200, 'disruption_cost': 1000},
        {'flight_id': 7, 'type': 'aircraft_change', 'new_aircraft': 'CC1',
         'scheduled_departure_time': datetime(2024, 1, 1, 12, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 13, 30),
         'airline_code': 'AS', 'flight_number': 404, 'departure_airport': 'SEA',
         'arrival_airport': 'YYZ', 'passenger_count': 100, 'disruption_cost': 7000},
        {'flight_id': 5, 'type': 'departure_delay', 'delay': 20,
         'scheduled_departure_time': datetime(2024, 1, 1, 8, 30),
         'scheduled_arrival_time': datetime(2024, 1, 1, 9, 30),
         'airline_code': 'B6', 'flight_number': 202, 'departure_airport': 'JFK',
         'arrival_airport': 'LAX', 'passenger_count': 160, 'disruption_cost': 8000},
        {'flight_id': 3, 'type': 'crew_delay', 'delay': 40,
         'scheduled_departure_time': datetime(2024, 1, 1, 11, 0),
         'scheduled_arrival_time': datetime(2024, 1, 1, 12, 0),
         'airline_code': 'UA', 'flight_number': 789, 'departure_airport': 'ORD',
         'arrival_airport': 'DFW', 'passenger_count': 250, 'disruption_cost': 11000}
    ]
    gemini_agent.process_realtime_updates(updates)

Attempting to get Gemini API key...
Model: models/gemini-2.5-pro-preview-05-06



Flight 1: Predicted Delay = 2.38
Flight 2: Predicted Delay = 13.6
Flight 3: Predicted Delay = 4.08
Flight 4: Predicted Delay = 8.16
Flight 5: Predicted Delay = 6.46
Flight 6: Predicted Delay = 5.1
Flight 7: Predicted Delay = 22.78
Flight 8: Predicted Delay = 5.1
--- Gemini Prompt (models/gemini-2.5-pro-preview-05-06): ---

                    Analyze the potential impact of a 13.6 minute disruption to Flight DL 456
                    (Departure: BOS, Arrival: ORD, Scheduled Departure: 2024-01-01 09:00:00).

                    Consider the following factors:

                    **Flight Details:**
                    -   Aircraft type: 737
                    -   Passenger count: 140
                    -   Estimated disruption cost: 7000

                    **Disruption Details:**
                    -   Disruption type: crew_delay
                    -   Delay (minutes): 13.6

                    **C